In [3]:
from surprise import Reader, Dataset, KNNWithMeans, accuracy
from surprise.model_selection import train_test_split
import pandas as pd
import numpy as np

In [4]:
# 데이터 로드 및 전처리
df = pd.read_csv('../df.csv')
df.reset_index(inplace=True, drop=True)
df = df[['user_id', 'movie_id', 'rating']]
df = df[:10000]

# 평점 범위를 1~10으로 설정
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df, reader)

# 학습 및 테스트 데이터셋 분리
trainset, testset = train_test_split(data, test_size=0.3, random_state=10)

# KNN 모델 학습
algo = KNNWithMeans(k=20, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

# 테스트 데이터셋으로 예측 및 RMSE 계산
test_pred = algo.test(testset)
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

# 특정 사용자에 대한 영화 추천 함수
def get_recommendations(algo, user_id, df, n=20):
    # 사용자가 평가하지 않은 영화 목록 생성
    user_rated_movies = df[df['user_id'] == user_id]['movie_id'].tolist()
    all_movies = df['movie_id'].unique()
    unrated_movies = [movie for movie in all_movies if movie not in user_rated_movies]
    
    # 예상 평점 계산
    predictions = [algo.predict(user_id, movie) for movie in unrated_movies]
    predictions.sort(key=lambda x: x.est, reverse=True) #예상 평점 기준 정렬
    
    # 상위 n개의 영화 추천
    top_n_recommendations = predictions[:n]
    return [(pred.iid, pred.est) for pred in top_n_recommendations] # (영화id, 예상 평점)

# 사용자 ID에 대한 추천 영화 목록 출력
user_id = 2  
recommended_movies = get_recommendations(algo, user_id, df, n=20)
print("추천된 영화 목록:")
for movie_id, est_rating in recommended_movies:
    print(f"영화 ID: {movie_id}, 예상 평점: {est_rating}")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Item-based Model : Test Set
RMSE: 2.6305
추천된 영화 목록:
영화 ID: 10002, 예상 평점: 10
영화 ID: 100069, 예상 평점: 10
영화 ID: 10008, 예상 평점: 10
영화 ID: 10016, 예상 평점: 10
영화 ID: 10022, 예상 평점: 10
영화 ID: 10024, 예상 평점: 10
영화 ID: 10038, 예상 평점: 10
영화 ID: 10047, 예상 평점: 10
영화 ID: 100642, 예상 평점: 10
영화 ID: 10070, 예상 평점: 10
영화 ID: 10071, 예상 평점: 10
영화 ID: 10072, 예상 평점: 10
영화 ID: 10040, 예상 평점: 9.969215291750503
영화 ID: 100730, 예상 평점: 9.948508903668742
영화 ID: 10046, 예상 평점: 9.923660714285715
영화 ID: 100825, 예상 평점: 9.919354838709678
영화 ID: 10050, 예상 평점: 9.853246753246754
영화 ID: 100399, 예상 평점: 9.85135135135135
영화 ID: 10014, 예상 평점: 9.82943722943723
영화 ID: 10033, 예상 평점: 9.814285714285715
